[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Vedant2100/w26/blob/main/CS%20226/vegetation_anomalies_project.ipynb)

# Scalable Analysis of Vegetation Anomalies Preceding Plant Disease Outbreaks

**Project:** Distributed Analysis using Apache Spark  
**Team Size:** 5  
**Platform:** Spark (Databricks or EMR)  
**Storage:** S3 or HDFS  
**Primary Data:** Sentinel-2 L2A, Landsat C2 L2  

---

## Project Overview

### Goal
Analyze large-scale satellite imagery to identify vegetation anomalies that may precede plant disease outbreaks using distributed computing frameworks.

### Key Deliverables
- Clean, forest-masked, partitioned reflectance dataset
- NDVI time series and historical baselines
- Anomaly detection catalog with lead-lag correlation analysis
- Scalability evaluation report
- Interactive visualizations and final presentation

### 4-Week Timeline
- **Week 1:** Data Infrastructure & Preprocessing
- **Week 2:** Vegetation Indices & Baseline Modeling
- **Week 3:** Anomaly Detection & Validation
- **Week 4:** Scaling, Visualization & Finalization

## Environment Setup and Imports

In [ ]:
# Core PySpark Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, count, when, avg, sum as _sum
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
from pyspark.sql.window import Window

# Geospatial Processing
try:
    import geopandas as gpd
    from sedona.spark import SedonaContext
    from sedona.sql import st_constructors as ST
    print("✓ Sedona/GeoSpark loaded")
except ImportError:
    print("! Install Sedona: pip install apache-sedona")

# Raster Processing
try:
    import rasterio
    from rasterio.features import rasterize
    print("✓ Rasterio loaded")
except ImportError:
    print("! Install rasterio: pip install rasterio")

# Visualization and Analysis
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Performance Tracking
import time
from typing import Dict, List, Tuple

print("\n=== Environment Setup Complete ===")

## Storage Structure Configuration

```
s3://your-bucket/
├── raw/                  # Raw satellite imagery
├── processed/            # Quality-filtered data
│   └── forest_masked/    # Forest-only pixels
├── ndvi/                 # NDVI time series
├── baseline/             # Historical baseline statistics
├── anomalies/            # Detected anomalies
└── reference/            # Outbreak reference data
```

In [ ]:
# Configuration
BUCKET_NAME = "your-bucket-name"  # TODO: Update with your S3 bucket
BASE_PATH = f"s3://{BUCKET_NAME}/"

PATHS = {
    'raw': f"{BASE_PATH}raw/",
    'processed': f"{BASE_PATH}processed/",
    'forest_masked': f"{BASE_PATH}processed/forest_masked/",
    'ndvi': f"{BASE_PATH}ndvi/",
    'baseline': f"{BASE_PATH}baseline/",
    'anomalies': f"{BASE_PATH}anomalies/",
    'reference': f"{BASE_PATH}reference/"
}

# Region of Interest (example bounding box)
ROI = {
    'min_lat': 35.0,
    'max_lat': 45.0,
    'min_lon': -125.0,
    'max_lon': -110.0
}

# Time range
START_YEAR = 2015
END_YEAR = 2024

print("Configuration loaded successfully")

---
# WEEK 1: Data Infrastructure & Preprocessing
🎯 **Goal:** Produce a clean, forest-masked, partitioned reflectance dataset ready for NDVI computation

## Epic 5 — Story 5.1: Spark Cluster Configuration
**Owner:** DevOps Lead

In [ ]:
# Initialize Spark Session with optimized configuration
spark = SparkSession.builder \
    .appName("VegetationAnomalyDetection") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.default.parallelism", "200") \
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB") \
    .getOrCreate()

# Enable Delta Lake support if available
# .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
# .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

print(f"Spark version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"Workers: {spark.sparkContext.defaultParallelism}")

In [ ]:
# Cluster Validation Test
print("Testing cluster with simple read/write benchmark...")
start_time = time.time()

# Create test data
test_data = spark.range(0, 10000000).selectExpr("id", "rand() as value")

# Write test
test_path = f"{BASE_PATH}test/benchmark"
test_data.write.mode("overwrite").parquet(test_path)

# Read test
read_data = spark.read.parquet(test_path)
result = read_data.agg({"value": "avg"}).collect()

elapsed = time.time() - start_time
print(f"✓ Benchmark completed in {elapsed:.2f} seconds")
print(f"✓ Partitions: {read_data.rdd.getNumPartitions()}")

# TODO: Document cluster configuration in separate file

## Epic 1 — Story 1.1: Bulk Ingest Surface Reflectance
**Owner:** Dev1

In [ ]:
# Define metadata schema for satellite imagery
metadata_schema = StructType([
    StructField("tile_id", StringType(), False),
    StructField("satellite", StringType(), False),
    StructField("date", TimestampType(), False),
    StructField("year", IntegerType(), False),
    StructField("month", IntegerType(), False),
    StructField("region", StringType(), False),
    StructField("path", StringType(), False),
    StructField("red_band", StringType(), True),
    StructField("nir_band", StringType(), True),
    StructField("qa_band", StringType(), True)
])

def ingest_reflectance_data(years: List[int], roi: Dict) -> None:
    """
    Ingest surface reflectance data from Sentinel-2 and Landsat.
    
    TODO: Implement actual data ingestion logic:
    - Query satellite data APIs (e.g., Google Earth Engine, AWS Open Data)
    - Download tile references
    - Convert to Spark DataFrames
    - Partition by year, month, region
    - Write to Parquet in /raw/
    """
    start_time = time.time()
    
    # Example: Create dummy metadata (replace with actual ingestion)
    print(f"Ingesting data for years {years[0]}-{years[-1]}...")
    print(f"ROI: Lat({roi['min_lat']}, {roi['max_lat']}), Lon({roi['min_lon']}, {roi['max_lon']})")
    
    # TODO: Implement data retrieval logic here
    # Example structure:
    # for year in years:
    #     for month in range(1, 13):
    #         tiles = query_satellite_api(year, month, roi)
    #         df = create_spark_dataframe(tiles)
    #         df.write.partitionBy("year", "month", "region") \
    #           .parquet(f"{PATHS['raw']}/reflectance")
    
    elapsed = time.time() - start_time
    print(f"\n✓ Ingestion completed in {elapsed:.2f} seconds")
    # TODO: Log number of tiles ingested, total size, runtime

# Run ingestion
# ingest_reflectance_data(list(range(START_YEAR, END_YEAR + 1)), ROI)

## Epic 1 — Story 1.2: Distributed Quality Filtering
**Owner:** Dev2

In [ ]:
def apply_qa_filtering(input_path: str, output_path: str) -> Dict[str, float]:
    """
    Apply distributed quality filtering to remove clouds and shadows.
    
    TODO: Implement QA band masking:
    - Load raw reflectance dataset
    - Identify QA band (varies by satellite)
    - Decode bit flags for cloud/shadow
    - Apply pixel-level mask
    - Compute statistics on removed pixels
    """
    start_time = time.time()
    
    # Load data
    print(f"Loading data from {input_path}...")
    # df = spark.read.parquet(input_path)
    
    # TODO: Implement QA masking logic
    # Example for Landsat:
    # - Bit 3: Cloud
    # - Bit 4: Cloud Shadow
    # 
    # def is_clear_pixel(qa_value):
    #     return (qa_value & (1 << 3)) == 0 and (qa_value & (1 << 4)) == 0
    # 
    # filtered_df = df.filter(is_clear_pixel(col("qa_band")))
    
    # Compute statistics
    # total_pixels = df.count()
    # clean_pixels = filtered_df.count()
    # percent_removed = ((total_pixels - clean_pixels) / total_pixels) * 100
    
    # Save cleaned data
    # filtered_df.write.mode("overwrite").parquet(output_path)
    
    elapsed = time.time() - start_time
    print(f"✓ Quality filtering completed in {elapsed:.2f} seconds")
    
    return {
        'runtime_seconds': elapsed,
        # 'total_pixels': total_pixels,
        # 'clean_pixels': clean_pixels,
        # 'percent_removed': percent_removed
    }

# Run filtering
# qa_stats = apply_qa_filtering(PATHS['raw'], PATHS['processed'])
# print(json.dumps(qa_stats, indent=2))

## Epic 1 — Story 1.3: Forest Masking via ESA WorldCover
**Owner:** Dev3

In [ ]:
def apply_forest_mask(input_path: str, worldcover_path: str, output_path: str) -> Dict:
    """
    Apply forest mask using ESA WorldCover dataset.
    
    TODO: Implement forest masking:
    - Ingest ESA WorldCover (10m resolution land cover)
    - Extract forest class code (typically code 10)
    - Perform spatial join with reflectance data
    - Align resolutions (reproject if needed)
    - Filter to forest-only pixels
    - Save to /processed/forest_masked/
    """
    start_time = time.time()
    
    print("Loading reflectance data...")
    # reflectance_df = spark.read.parquet(input_path)
    
    print("Loading ESA WorldCover land cover data...")
    # worldcover_df = spark.read.format("raster").load(worldcover_path)
    
    # TODO: Extract forest pixels (class code 10 in ESA WorldCover)
    # forest_mask = worldcover_df.filter(col("land_cover") == 10)
    
    # TODO: Spatial join
    # from sedona.sql.st_functions import ST_Intersects
    # masked_df = reflectance_df.join(
    #     forest_mask,
    #     ST_Intersects(reflectance_df.geometry, forest_mask.geometry)
    # )
    
    # Save masked data
    # masked_df.write.mode("overwrite").parquet(output_path)
    
    # Validation
    # total_pixels = reflectance_df.count()
    # forest_pixels = masked_df.count()
    # percent_forest = (forest_pixels / total_pixels) * 100
    
    elapsed = time.time() - start_time
    print(f"✓ Forest masking completed in {elapsed:.2f} seconds")
    # print(f"✓ Forest coverage: {percent_forest:.1f}%")
    
    return {
        'runtime_seconds': elapsed,
        # 'total_pixels': total_pixels,
        # 'forest_pixels': forest_pixels,
        # 'percent_forest': percent_forest
    }

# Run forest masking
# mask_stats = apply_forest_mask(
#     PATHS['processed'],
#     "s3://esa-worldcover/v100/",
#     PATHS['forest_masked']
# )

---
# WEEK 2: Vegetation Indices & Baseline Modeling
🎯 **Goal:** Generate NDVI time series and compute historical baselines

## Epic 2 — Story 2.1: Scalable NDVI Computation
**Owner:** Dev2

In [ ]:
def compute_ndvi(input_path: str, output_path: str) -> Dict:
    """
    Compute NDVI (Normalized Difference Vegetation Index) across all tiles.
    
    Formula: NDVI = (NIR - Red) / (NIR + Red)
    
    TODO: Implement distributed NDVI computation:
    - Load forest-masked reflectance data
    - Extract Red and NIR bands
    - Apply NDVI formula element-wise
    - Handle division by zero
    - Partition by year, month, region
    - Save to /ndvi/
    """
    start_time = time.time()
    
    print(f"Loading forest-masked data from {input_path}...")
    # df = spark.read.parquet(input_path)
    
    # TODO: Compute NDVI
    # from pyspark.sql.functions import when
    # 
    # ndvi_df = df.withColumn(
    #     "ndvi",
    #     when(
    #         (col("nir") + col("red")) == 0,
    #         None
    #     ).otherwise(
    #         (col("nir") - col("red")) / (col("nir") + col("red"))
    #     )
    # )
    
    # Filter valid NDVI range [-1, 1]
    # ndvi_df = ndvi_df.filter((col("ndvi") >= -1) & (col("ndvi") <= 1))
    
    # Save with partitioning
    # ndvi_df.write \
    #     .partitionBy("year", "month", "region") \
    #     .mode("overwrite") \
    #     .parquet(output_path)
    
    elapsed = time.time() - start_time
    print(f"✓ NDVI computation completed in {elapsed:.2f} seconds")
    
    # Performance metrics
    # num_partitions = ndvi_df.rdd.getNumPartitions()
    # total_records = ndvi_df.count()
    
    return {
        'runtime_seconds': elapsed,
        # 'num_partitions': num_partitions,
        # 'total_records': total_records
    }

# Run NDVI computation
# ndvi_stats = compute_ndvi(PATHS['forest_masked'], PATHS['ndvi'])

## Epic 2 — Story 2.2: Optional Multi-Index Exploration
**Owner:** Dev4

In [ ]:
def compute_additional_indices(df):
    """
    Compute additional vegetation indices for comparison.
    
    EVI (Enhanced Vegetation Index):
    EVI = 2.5 * ((NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1))
    
    NBR (Normalized Burn Ratio):
    NBR = (NIR - SWIR) / (NIR + SWIR)
    
    TODO: Implement and compare multiple indices
    """
    
    # EVI computation
    # evi_df = df.withColumn(
    #     "evi",
    #     2.5 * ((col("nir") - col("red")) / 
    #            (col("nir") + 6*col("red") - 7.5*col("blue") + 1))
    # )
    
    # NBR computation (requires SWIR band)
    # nbr_df = df.withColumn(
    #     "nbr",
    #     (col("nir") - col("swir")) / (col("nir") + col("swir"))
    # )
    
    # TODO: Compute correlation between indices
    # TODO: Visualize distributions
    # TODO: Decide which to include in final analysis
    
    pass

# Optional: Run for exploratory analysis
# compute_additional_indices(ndvi_df)

## Epic 3 — Story 3.1: Historical Baseline Construction
**Owner:** Dev1

In [ ]:
def compute_baseline(input_path: str, output_path: str) -> None:
    """
    Compute historical baseline NDVI statistics by month.
    
    TODO: Implement baseline construction:
    - Load NDVI time series
    - Group by region, month (across all years)
    - Compute mean and standard deviation
    - Create seasonal baseline model
    - Save to /baseline/
    """
    start_time = time.time()
    
    print(f"Loading NDVI data from {input_path}...")
    # ndvi_df = spark.read.parquet(input_path)
    
    # TODO: Compute baseline statistics
    # baseline_df = ndvi_df.groupBy("region", "month") \
    #     .agg(
    #         mean("ndvi").alias("mean_ndvi"),
    #         stddev("ndvi").alias("stddev_ndvi"),
    #         count("*").alias("sample_count")
    #     )
    
    # Save baseline
    # baseline_df.write.mode("overwrite").parquet(output_path)
    
    elapsed = time.time() - start_time
    print(f"✓ Baseline computed in {elapsed:.2f} seconds")
    
    # TODO: Visualize seasonal curves

# Run baseline computation
# compute_baseline(PATHS['ndvi'], PATHS['baseline'])

In [ ]:
# Visualization: Seasonal NDVI Baseline
def plot_seasonal_baseline(baseline_path: str, region: str = None):
    """
    Plot seasonal NDVI trends for selected region.
    """
    # baseline_pdf = spark.read.parquet(baseline_path).toPandas()
    
    # if region:
    #     baseline_pdf = baseline_pdf[baseline_pdf['region'] == region]
    
    # plt.figure(figsize=(12, 6))
    # plt.plot(baseline_pdf['month'], baseline_pdf['mean_ndvi'], marker='o', label='Mean NDVI')
    # plt.fill_between(
    #     baseline_pdf['month'],
    #     baseline_pdf['mean_ndvi'] - baseline_pdf['stddev_ndvi'],
    #     baseline_pdf['mean_ndvi'] + baseline_pdf['stddev_ndvi'],
    #     alpha=0.3, label='±1 Std Dev'
    # )
    # plt.xlabel('Month')
    # plt.ylabel('NDVI')
    # plt.title(f'Seasonal NDVI Baseline: {region or "All Regions"}')
    # plt.legend()
    # plt.grid(True, alpha=0.3)
    # plt.show()
    
    pass

# plot_seasonal_baseline(PATHS['baseline'])

---
# WEEK 3: Anomaly Detection & Validation
🎯 **Goal:** Identify vegetation deviations and compare against outbreak timing

## Epic 3 — Story 3.2: Distributed Anomaly Identification
**Owner:** Dev3

In [ ]:
def detect_anomalies(ndvi_path: str, baseline_path: str, output_path: str, 
                     threshold: float = -2.0) -> None:
    """
    Detect anomalous NDVI values using Z-score method.
    
    Z = (NDVI_current - Mean_baseline) / Std_baseline
    Anomaly if Z < threshold (e.g., -2.0)
    
    TODO: Implement anomaly detection:
    - Load NDVI and baseline data
    - Join on region and month
    - Compute Z-scores
    - Flag anomalies
    - Aggregate anomalous area per region/month
    - Save to /anomalies/
    """
    start_time = time.time()
    
    print("Loading NDVI and baseline data...")
    # ndvi_df = spark.read.parquet(ndvi_path)
    # baseline_df = spark.read.parquet(baseline_path)
    
    # TODO: Join and compute Z-scores
    # joined_df = ndvi_df.join(
    #     baseline_df,
    #     ["region", "month"],
    #     "left"
    # )
    # 
    # anomaly_df = joined_df.withColumn(
    #     "z_score",
    #     (col("ndvi") - col("mean_ndvi")) / col("stddev_ndvi")
    # ).withColumn(
    #     "is_anomaly",
    #     when(col("z_score") < threshold, 1).otherwise(0)
    # )
    
    # TODO: Aggregate anomalies
    # anomaly_summary = anomaly_df.groupBy("region", "year", "month") \
    #     .agg(
    #         _sum("is_anomaly").alias("anomaly_count"),
    #         count("*").alias("total_pixels"),
    #         (100.0 * _sum("is_anomaly") / count("*")).alias("percent_anomalous")
    #     )
    
    # Save anomalies
    # anomaly_df.write.mode("overwrite").parquet(f"{output_path}/pixels")
    # anomaly_summary.write.mode("overwrite").parquet(f"{output_path}/summary")
    
    elapsed = time.time() - start_time
    print(f"✓ Anomaly detection completed in {elapsed:.2f} seconds")
    print(f"Threshold: Z < {threshold}")

# Run anomaly detection
# detect_anomalies(PATHS['ndvi'], PATHS['baseline'], PATHS['anomalies'])

## Epic 4 — Story 4.1: Outbreak Data Integration
**Owner:** Dev4

In [ ]:
def ingest_outbreak_data(outbreak_shapefile: str, output_path: str) -> None:
    """
    Ingest and standardize plant disease outbreak records.
    
    TODO: Implement outbreak data ingestion:
    - Load outbreak shapefiles or CSV
    - Standardize fields: date, region, disease_type, severity
    - Convert to Spark spatial DataFrame
    - Save to /reference/outbreaks/
    """
    
    # Example using GeoPandas + Spark
    # gdf = gpd.read_file(outbreak_shapefile)
    # 
    # # Standardize columns
    # gdf = gdf.rename(columns={
    #     'detection_date': 'date',
    #     'location': 'region',
    #     'disease': 'disease_type'
    # })
    # 
    # # Convert to Spark
    # outbreak_df = spark.createDataFrame(gdf)
    # 
    # # Save
    # outbreak_df.write.mode("overwrite").parquet(output_path)
    
    print(f"✓ Outbreak data ingested to {output_path}")

# TODO: Provide path to outbreak data
# ingest_outbreak_data("/path/to/outbreak.shp", PATHS['reference'])

## Epic 4 — Story 4.2: Lead-Lag Correlation Analysis
**Owner:** Dev5

In [ ]:
def analyze_lead_lag(anomaly_path: str, outbreak_path: str) -> pd.DataFrame:
    """
    Analyze temporal relationship between anomalies and outbreaks.
    
    TODO: Implement lead-lag analysis:
    - Join anomalies with outbreak regions
    - Compute time difference (anomaly spike -> outbreak detection)
    - Calculate mean lead time and distribution
    - Perform statistical test (t-test or Mann-Whitney)
    - Generate summary tables
    """
    
    print("Loading anomaly and outbreak data...")
    # anomaly_df = spark.read.parquet(f"{anomaly_path}/summary")
    # outbreak_df = spark.read.parquet(outbreak_path)
    
    # TODO: Spatial-temporal join
    # joined_df = anomaly_df.join(
    #     outbreak_df,
    #     (anomaly_df.region == outbreak_df.region) &
    #     (anomaly_df.date <= outbreak_df.date),
    #     "inner"
    # )
    
    # TODO: Compute lead time
    # from pyspark.sql.functions import datediff
    # lead_lag_df = joined_df.withColumn(
    #     "lead_days",
    #     datediff(col("outbreak_date"), col("anomaly_date"))
    # )
    
    # TODO: Statistical analysis
    # results = lead_lag_df.groupBy("disease_type") \
    #     .agg(
    #         mean("lead_days").alias("mean_lead_days"),
    #         stddev("lead_days").alias("std_lead_days"),
    #         count("*").alias("n_events")
    #     )
    
    # results_pdf = results.toPandas()
    # return results_pdf
    
    pass

# Run lead-lag analysis
# lead_lag_results = analyze_lead_lag(PATHS['anomalies'], PATHS['reference'])
# print(lead_lag_results)

---
# WEEK 4: Scaling, Visualization & Finalization

## Epic 5 — Story 5.2: Scalability Evaluation
**Owner:** DevOps + Dev1

In [ ]:
def benchmark_scalability():
    """
    Evaluate horizontal scaling efficiency.
    
    TODO: Run NDVI computation on:
    - 1 year of data
    - 3 years of data
    - Full dataset (2015-2024)
    
    Record: runtime, memory, CPU usage, data volume
    Plot scaling curves
    """
    
    results = []
    
    for year_range in [1, 3, 10]:
        start_time = time.time()
        
        # TODO: Run NDVI pipeline on subset
        # compute_ndvi(filtered_data, output)
        
        elapsed = time.time() - start_time
        
        results.append({
            'years': year_range,
            'runtime_seconds': elapsed,
            # 'memory_gb': ...,
            # 'data_volume_gb': ...
        })
    
    # Plot results
    results_df = pd.DataFrame(results)
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))
    
    # Runtime vs Data Size
    ax[0].plot(results_df['years'], results_df['runtime_seconds'], marker='o')
    ax[0].set_xlabel('Years of Data')
    ax[0].set_ylabel('Runtime (seconds)')
    ax[0].set_title('Scaling: Runtime vs Data Volume')
    ax[0].grid(True, alpha=0.3)
    
    # Efficiency
    # ax[1].plot(...)
    
    plt.tight_layout()
    # plt.savefig('scalability_report.png', dpi=150)
    plt.show()
    
    return results_df

# Run scalability benchmark
# scalability_results = benchmark_scalability()

## Epic 6 — Story 6.1: Exploratory Visualization
**Owner:** Dev5

In [ ]:
def create_anomaly_heatmap(anomaly_path: str, outbreak_path: str, region: str = None):
    """
    Create temporal heatmap of anomalies with outbreak overlays.
    """
    
    # Load data
    # anomaly_summary = spark.read.parquet(f"{anomaly_path}/summary").toPandas()
    # outbreaks = spark.read.parquet(outbreak_path).toPandas()
    
    # if region:
    #     anomaly_summary = anomaly_summary[anomaly_summary['region'] == region]
    #     outbreaks = outbreaks[outbreaks['region'] == region]
    
    # Create heatmap
    # pivot_table = anomaly_summary.pivot_table(
    #     values='percent_anomalous',
    #     index='month',
    #     columns='year',
    #     aggfunc='mean'
    # )
    
    # plt.figure(figsize=(14, 8))
    # sns.heatmap(pivot_table, cmap='RdYlGn_r', annot=True, fmt='.1f', 
    #             cbar_kws={'label': '% Anomalous Area'})
    # 
    # # Overlay outbreak markers
    # for _, outbreak in outbreaks.iterrows():
    #     plt.plot(outbreak['year'], outbreak['month'], 'X', 
    #              markersize=15, color='black', label='Outbreak')
    # 
    # plt.title(f'NDVI Anomalies and Disease Outbreaks: {region or "All Regions"}')
    # plt.xlabel('Year')
    # plt.ylabel('Month')
    # plt.tight_layout()
    # plt.show()
    
    pass

# Create visualization
# create_anomaly_heatmap(PATHS['anomalies'], PATHS['reference'])

In [ ]:
def create_time_series_dashboard(ndvi_path: str, anomaly_path: str, outbreak_path: str):
    """
    Create interactive time series plots.
    
    TODO: Consider using Plotly or Bokeh for interactivity
    """
    
    # Example static visualization
    # ndvi_ts = spark.read.parquet(ndvi_path).toPandas()
    # anomalies = spark.read.parquet(f"{anomaly_path}/summary").toPandas()
    # outbreaks = spark.read.parquet(outbreak_path).toPandas()
    
    # Plot time series with anomalies highlighted
    # fig, ax = plt.subplots(figsize=(16, 6))
    # 
    # ax.plot(ndvi_ts['date'], ndvi_ts['ndvi'], label='NDVI', alpha=0.7)
    # ax.scatter(
    #     anomalies['date'],
    #     anomalies['ndvi'],
    #     c='red',
    #     label='Anomaly',
    #     s=50,
    #     alpha=0.6
    # )
    # 
    # for _, outbreak in outbreaks.iterrows():
    #     ax.axvline(outbreak['date'], color='orange', linestyle='--', 
    #                alpha=0.5, label='Outbreak')
    # 
    # ax.set_xlabel('Date')
    # ax.set_ylabel('NDVI')
    # ax.set_title('NDVI Time Series with Anomalies and Outbreaks')
    # ax.legend()
    # ax.grid(True, alpha=0.3)
    # plt.show()
    
    pass

# create_time_series_dashboard(PATHS['ndvi'], PATHS['anomalies'], PATHS['reference'])

## Epic 6 — Story 6.2: Final Report & Presentation
**All Members**

### Report Sections

1. **Executive Summary**
   - Project goals and motivation
   - Key findings
   - Scalability results

2. **Data Pipeline Architecture**
   - Data sources and volume
   - Spark cluster configuration
   - Storage structure and partitioning strategy
   - Processing stages (ingestion → masking → NDVI → anomalies)

3. **Methodology**
   - Quality filtering approach
   - Forest masking using ESA WorldCover
   - NDVI computation and baseline modeling
   - Anomaly detection algorithm (Z-score threshold)

4. **Results**
   - NDVI time series patterns
   - Anomaly frequency and spatial distribution
   - Lead-lag correlation with outbreaks
   - Statistical significance tests

5. **Scalability Analysis**
   - Runtime vs data volume
   - Horizontal scaling efficiency
   - Memory and CPU utilization
   - Bottleneck identification

6. **Limitations**
   - **No predictive claims**: This is exploratory correlation analysis only
   - Cloud contamination despite QA filtering
   - Temporal resolution constraints
   - Outbreak data completeness

7. **Future Work**
   - Scale to petabyte-level global analysis
   - Incorporate additional data sources (weather, soil)
   - Machine learning for predictive modeling
   - Real-time anomaly detection pipeline

8. **Conclusions**
   - Summary of findings
   - Implications for early warning systems
   - Distributed computing lessons learned

---
## Performance Metrics Tracking

In [ ]:
# Template for tracking all metrics across pipeline stages
performance_metrics = {
    'ingestion': {
        'runtime_seconds': None,
        'tiles_ingested': None,
        'data_volume_gb': None
    },
    'qa_filtering': {
        'runtime_seconds': None,
        'pixels_removed_pct': None
    },
    'forest_masking': {
        'runtime_seconds': None,
        'forest_coverage_pct': None
    },
    'ndvi_computation': {
        'runtime_seconds': None,
        'num_partitions': None,
        'total_records': None
    },
    'anomaly_detection': {
        'runtime_seconds': None,
        'anomalies_detected': None
    },
    'scalability': {
        '1_year_runtime': None,
        '3_year_runtime': None,
        'full_dataset_runtime': None
    }
}

# Save metrics
# with open('performance_metrics.json', 'w') as f:
#     json.dump(performance_metrics, f, indent=2)

---
## Risk Mitigation Checklist

### If data volume is too large:
- ✅ Reduce ROI but maintain distributed framework
- ✅ Focus on 2-3 years instead of full decade
- ✅ Sample tiles instead of processing all

### If anomaly signal is weak:
- ✅ Emphasize exploratory findings
- ✅ Document methodology thoroughly
- ✅ Discuss data quality and limitations

### If cluster is unstable:
- ✅ Reduce number of workers
- ✅ Optimize partition sizes
- ✅ Increase executor memory
- ✅ Enable checkpointing for long jobs

### If processing is too slow:
- ✅ Profile code to identify bottlenecks
- ✅ Optimize shuffle operations
- ✅ Use broadcast joins for small datasets
- ✅ Cache intermediate results strategically

---
## Team Collaboration Notes

**Weekly Standups:**
- Monday: Week planning and task assignment
- Friday: Progress review and blocker discussion

**Code Review:**
- All major pipeline changes require 1 reviewer approval
- Focus on efficiency, correctness, documentation

**Documentation:**
- Keep this notebook updated with latest code
- Document all parameter choices and thresholds
- Record runtime metrics for each stage

**Communication Channels:**
- Slack for quick questions
- GitHub Issues for bugs and feature requests
- Weekly meetings for strategic decisions

In [ ]:
# Cleanup: Stop Spark session when done
# spark.stop()
# print("Spark session stopped")